<a href="https://colab.research.google.com/github/aedhcarrick/nacho-diffusion/blob/dev/nacho_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title -=Setup =-
!nvidia-smi -L
%cd /content
!echo -= Download Repo =-
!git clone https://github.com/aedhcarrick/nacho-diffusion
%cd /content/nacho-diffusion
!git checkout dev
!git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

!echo -= Download Model =-
!wget -c https://huggingface.co/XpucT/Deliberate/resolve/main/deliberate_v2.ckpt -O ./models/checkpoints/model.ckpt


In [ ]:
#@title -= Nacho Generator =-
SAMPLER = "DDIM" #@param ["DDIM", "DPM", "PLMS"]
STEPS = 50 #@param {type:"slider", min:"1", max:"100", step:1}
GUIDANCE = 7.5 #@param {type:"slider", min:"1", max:"20", step:0.5}
SEED = 42 #@param {type:"integer"}
BATCH_SIZE = 1 #@param {type:"integer"}

from image_worker import ImageWorker

worker = ImageWorker()
job = {
    "sampler": SAMPLER,
    "steps": STEPS,
    "guidance": GUIDANCE,
    "seed": SEED,
    "batch_size": BATCH_SIZE,
}

worker.submit_job('txt2img', job)
worker.run()


In [ ]:
#@title -= Pytest =-
!python -m pytest